<div style="text-align: center;"> <h1> FURIA Know Your Fan </h1> </div>

<div style="text-align: center;"> <h3> Todos os pip, ferramentas e bibliotecas utilizadas </h3> </div>

- pip install python-dotenv ipywidgets cryptography

<div style="text-align: center;"> <h3> Chave API Oculta (Arquivo .env) </h3> </div>

In [ ]:
from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

chave_api = os.getenv('API_KEY')

<div style="text-align: center;"> <h3>Coleta de Dados Básicos e Interesses Relacionados à FURIA</h3> </div>
  
- **Formulário Inicial:** Criar um formulário em Jupyter Notebook (usando `ipywidgets` ou `Streamlit`) para capturar dados pessoais fundamentais: nome, CPF, endereço, data de nascimento, email etc. Validar formato de CPF (biblioteca `python-bcpf` ou validação via regex) e outros campos para evitar erros de digitação.  

- **Interesses e Atividades em e-sports:** Incluir no notebook seções ou perguntas interativas sobre interesses em e-sports, times preferidos (FURIA), jogos mais acompanhados, frequência de eventos assistidos, ingressos ou periféricos adquiridos no último ano. Utilizar `pandas` para estruturar as respostas em um DataFrame. Poder-se-á simular coleta de dados de APIs públicas de eventos (por exemplo, dados de torneios CS:GO) ou pedir ao usuário que importe seu histórico (como um CSV de compras) para preencher esse perfil de maneira realista.  

- **Compras Relacionadas:** Se for aplicável, permitir o upload de extratos simplificados ou listas de compras (como merch de teams e-sports). Usar `pandas` ou `openpyxl` para ler esses arquivos e filtrar itens de interesse (palavras-chave relacionadas a jogos, marcas de e-sports, etc).

In [ ]:
                                    # Rodando no Navegador

import http.server
import socketserver
import webbrowser
import json
import uuid
import os
import hashlib 

PORT = 8080
DATA_DIR = "form_data"

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_POST(self):
        if self.path == '/submit':
            content_length = int(self.headers['Content-Length'])
            post_data_bytes = self.rfile.read(content_length)

            # Decodificar os dados recebidos para string
            post_data_str = post_data_bytes.decode('utf-8')

            try:
                # Tentar carregar os dados como JSON
                dados_recebidos = json.loads(post_data_str)

                # Verificar se a chave 'cpf' existe
                if 'cpf' in dados_recebidos:
                    cpf_original = dados_recebidos['cpf']
                
                    cpf_hashed = hashlib.sha256(cpf_original.encode('utf-8')).hexdigest()
                    # Substituir o CPF original pelo hash
                    dados_recebidos['cpf'] = cpf_hashed
                else:
                     print("Aviso: Chave 'cpf' não encontrada nos dados recebidos.")

                # Gerar ID único
                user_id = str(uuid.uuid4())

                # Criar o diretório se não existir
                if not os.path.exists(DATA_DIR):
                    os.makedirs(DATA_DIR)

                # Caminho do arquivo para salvar
                file_path = os.path.join(DATA_DIR, f"{user_id}.json")

                # Salvar os dados modificados (com CPF hasheado) em JSON
                with open(file_path, 'w', encoding='utf-8') as f:
                    # Salvar o dicionário Python diretamente como JSON
                    json.dump(dados_recebidos, f, indent=4, ensure_ascii=False)

                print(f"\n✅ Dados processados (CPF com hash) salvos em: {file_path}")

                # Enviar resposta de sucesso
                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                self.wfile.write(json.dumps({'status': 'success', 'message': 'Dados recebidos e salvos com sucesso!'}).encode('utf-8'))

            except json.JSONDecodeError:
                print("\n❌ Erro: Não foi possível decodificar os dados JSON recebidos.")
                self.send_error(400, "Bad Request: Invalid JSON data")
            except Exception as e:
                print(f"\n❌ Erro ao processar a requisição: {e}")
                self.send_error(500, "Internal Server Error")

        else:
            super().do_GET()

def run_server():
    with socketserver.TCPServer(("", PORT), MyHandler) as httpd:
        print(f"Servidor rodando em http://localhost:{PORT}")
        webbrowser.open(f"http://localhost:{PORT}/form.html")
        httpd.serve_forever()

if __name__ == "__main__":
    run_server()

Servidor rodando em http://localhost:8080


127.0.0.1 - - [28/Apr/2025 17:27:33] "GET /form.html HTTP/1.1" 200 -



✅ Dados processados (CPF com hash) salvos em: form_data\8cf071a1-0212-475e-8e87-45d0959b3fb1.json


127.0.0.1 - - [28/Apr/2025 17:27:43] "POST /submit HTTP/1.1" 200 -


<div style="text-align: center;"> <h3>Validação de Identidade com Abordagem de IA</h3> </div>

- **Upload de Documentos:** Incluir um widget de upload de arquivos de imagem (RG, CNH ou passaporte) e, opcionalmente, uma selfie do usuário.  
- **OCR e Extração de Dados:** Utilizar uma biblioteca de OCR como `pytesseract` ou `easyocr` para extrair texto do documento. Comparar nome e CPF extraídos com os dados básicos fornecidos para consistência.  
- **Reconhecimento Facial:** Para fortalecer a validação, aplicar uma técnica de reconhecimento facial simples. Usar bibliotecas como `face_recognition` ou `OpenCV` com modelos pré-treinados para detectar rostos na selfie e na foto do documento, e então verificar se pertencem à mesma pessoa (por exemplo, comparando descritores faciais).  
- **Feedback de Validação:** Exibir no notebook resultados da validação (válido/inválido) com base na correspondência de texto e face. Fornecer mensagens orientativas caso haja inconsistências (ex: “CPF não confere com o documento enviado”).

### Instale as bibliotecas necessárias

- pytesseract (requer Tesseract OCR engine instalado separadamente)
 
- Pillow (dependência do pytesseract)
 
- face_recognition (requer dlib, pode ser complexo instalar em alguns sistemas)

- ipywidgets (para o widget de upload)

<br>

```pip install pytesseract Pillow face_recognition ipywidgets ```


### 1. Preparação e Upload de Documentos

Começamos pedindo ao usuário para carregar os arquivos necessários via widget. Precisaremos da imagem do documento completo para OCR e extração de dados, e possivelmente uma imagem separada da foto do documento e uma selfie para reconhecimento facial. Vamos criar widgets separados para clareza.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
import os

# Widgets de Upload
print("--- Upload de Documentos ---")

print("\nPor favor, carregue a IMAGEM COMPLETA do documento (RG, CNH, Passaporte):")
uploader_documento_completo = widgets.FileUpload(
    accept='.png,.jpg,.jpeg',
    multiple=False
)
display(uploader_documento_completo)

print("\nPor favor, carregue a IMAGEM SOMENTE da FOTO presente no documento:")
uploader_foto_documento = widgets.FileUpload(
    accept='.png,.jpg,.jpeg',
    multiple=False
)
display(uploader_foto_documento)

print("\nPor favor, carregue sua SELFIE:")
uploader_selfie = widgets.FileUpload(
    accept='.png,.jpg,.jpeg',
    multiple=False
)
display(uploader_selfie)

print("\nCarregue os arquivos usando os botões acima. As próximas etapas processarão os arquivos carregados.")

# Variáveis para armazenar as imagens PIL carregadas
img_documento_completo = None
img_foto_documento = None
img_selfie = None

# Função para carregar uma imagem de um widget de upload
def load_image_from_widget(uploader_widget):
    if uploader_widget.value:
        # Pega o primeiro arquivo (multiple=False)
        uploaded_file_info = uploader_widget.value[next(iter(uploader_widget.value))]
        file_content = uploaded_file_info['content']
        try:
            img = Image.open(io.BytesIO(file_content))
            print(f"'{uploaded_file_info['metadata']['name']}' carregado com sucesso.")
            return img
        except Exception as e:
            print(f"Erro ao abrir a imagem '{uploaded_file_info['metadata']['name']}': {e}")
            return None
    else:
        print("Nenhum arquivo carregado no widget.")
        return None

### 2. Processar Uploads e Preparar Imagens

Execute esta célula depois que todos os arquivos foram carregados nos widgets acima.

In [ ]:
print("--- Processando Imagens Carregadas ---")
clear_output(wait=True) # Limpa a saída anterior dos widgets para organização

# Carregar as imagens usando a função auxiliar
img_documento_completo = load_image_from_widget(uploader_documento_completo)
img_foto_documento = load_image_from_widget(uploader_foto_documento)
img_selfie = load_image_from_widget(uploader_selfie)

# Verificar se as imagens essenciais foram carregadas
if img_documento_completo and img_foto_documento and img_selfie:
    print("\nTodas as imagens essenciais foram carregadas. Próximas etapas: OCR e Reconhecimento Facial.")
    # Opcional: Exibir miniaturas das imagens carregadas para confirmação
    # from IPython.display import display as display_image
    # print("\nVisualização rápida:")
    # display_image(img_documento_completo.resize((100, int(100 * img_documento_completo.size[1] / img_documento_completo.size[0]))))
    # display_image(img_foto_documento.resize((100, int(100 * img_foto_documento.size[1] / img_foto_documento.size[0]))))
    # display_image(img_selfie.resize((100, int(100 * img_selfie.size[1] / img_selfie.size[0]))))

else:
    print("\nErro: Nem todas as imagens necessárias foram carregadas. Por favor, carregue todos os arquivos usando os widgets na célula anterior.")

### 3. OCR, Extração e Comparação de Dados Textuais

Definimos uma função para realizar o OCR na imagem do documento completo, extrair o texto, tentar encontrar Nome e CPF e compará-los com dados fornecidos (vamos simular esses dados fornecidos para o exemplo).

In [ ]:
import json
import base64
import io
from PIL import Image
import pytesseract
import re

# --- Carregar dados do JSON gerado pelo formulário ---
json_file = 'form_data/<user_id>.json'  # Atualize para o nome do arquivo salvo pelo servidor
with open(json_file, 'r', encoding='utf-8') as f:
    form_data = json.load(f)

# Extrair dados fornecidos
provided_data = {
    'nome': form_data.get('nome', ''),
    'cpf': form_data.get('cpf', '')
}

# Decodificar imagem do documento (base64) para PIL Image
foto_doc_b64 = form_data.get('foto_documento', '')
img_documento_completo = None
if foto_doc_b64:
    try:
        raw_bytes = base64.b64decode(foto_doc_b64)
        img_documento_completo = Image.open(io.BytesIO(raw_bytes))
    except Exception as e:
        print(f"Erro ao decodificar documento: {e}")
else:
    print("Campo 'foto_documento' ausente no JSON.")

# Função para limpar CPF

def clean_cpf(cpf_str):
    if isinstance(cpf_str, str):
        return cpf_str.replace('.', '').replace('-', '').strip()
    return ''

# Função de OCR e extração

def perform_ocr_and_extract_data(pil_image, provided_user_data):
    results = {
        'extracted_text': None,
        'extracted_cpf': None,
        'name_match': False,
        'cpf_match': False,
        'success': False,
        'message': ''
    }
    if pil_image is None:
        results['message'] = 'Imagem do documento não fornecida.'
        return results
    try:
        text = pytesseract.image_to_string(pil_image, lang='por')
        results['extracted_text'] = text

        # Extrair CPF
        m = re.search(r"\d{2,3}\.\d{3}\.\d{3}-?\d{2}", text)
        if m:
            results['extracted_cpf'] = m.group(0)

        # Comparar CPF
        if results['extracted_cpf'] and provided_user_data['cpf']:
            results['cpf_match'] = (clean_cpf(results['extracted_cpf']) == clean_cpf(provided_user_data['cpf']))

        # Comparar Nome (simplificado)
        name_parts = provided_user_data['nome'].lower().split()
        if len(name_parts) >= 2 and name_parts[0] in text.lower() and name_parts[-1] in text.lower():
            results['name_match'] = True

        results['success'] = True
        results['message'] = 'OCR e comparação concluídos.'
    except Exception as e:
        results['message'] = f'Erro no OCR: {e}'
    return results

# Executar OCR e exibir resultados
ocr_data = perform_ocr_and_extract_data(img_documento_completo, provided_data)
print("Resultado OCR:", ocr_data)


### 4. Reconhecimento e Comparação Facial

Agora, definimos uma função para lidar com a comparação facial entre a foto do documento e a selfie

In [ ]:
# -*- coding: utf-8 -*-
import face_recognition
import numpy as np

def perform_face_comparison(img_documento_foto, img_selfie_foto):
    """
    Compara a face na foto do documento com a face na selfie.

    Retorna:
        dict: Resultados da comparação facial ({'face_match', 'distance', 'success', 'message'})
    """
    results = {
        'face_match': False,
        'distance': None,
        'success': False, # Indica se o processo de comparação ocorreu sem erros graves
        'message': 'Comparação facial pendente.'
    }

    if img_documento_foto is None or img_selfie_foto is None:
        results['message'] = 'Erro: Imagens da foto do documento e/ou selfie não fornecidas para comparação facial.'
        return results

    try:
        print("Iniciando comparação facial...")

        # Converter imagens PIL para array numpy (face_recognition trabalha com numpy)
        img_doc_np = np.array(img_documento_foto.convert('RGB'))
        img_selfie_np = np.array(img_selfie_foto.convert('RGB'))


        # Encontrar os 'encodings' (características únicas) das faces
        print("Extraindo encodings da foto do documento...")
        document_face_encodings = face_recognition.face_encodings(img_doc_np)

        print("Extraindo encodings da selfie...")
        selfie_face_encodings = face_recognition.face_encodings(img_selfie_np)

        if not document_face_encodings:
            results['message'] = "Erro: Não foi possível detectar um rosto na foto do documento."
            print(results['message'])
            return results
        if not selfie_face_encodings:
            results['message'] = "Erro: Não foi possível detectar um rosto na selfie."
            print(results['message'])
            return results

        # Comparar o primeiro rosto encontrado em cada imagem
        known_face_encoding = document_face_encodings[0]
        unknown_face_encoding = selfie_face_encodings[0]

        # Comparar faces. Retorna True se for um match (abaixo de um limiar interno, padrão 0.6)
        results['face_match'] = face_recognition.compare_faces([known_face_encoding], unknown_face_encoding)[0]

        # Calcular a distância facial. Menor distância = maior similaridade.
        # O limiar de 0.6 mencionado antes é a distância padrão para `compare_faces`.
        results['distance'] = face_recognition.face_distance([known_face_encoding], unknown_face_encoding)[0]

        results['success'] = True
        results['message'] = 'Comparação facial concluída.'
        print(results['message'])

    except Exception as e:
        results['message'] = f"Erro durante o reconhecimento facial: {e}"
        results['success'] = False
        print(results['message'])

    return results

# --- Executar a etapa de Comparação Facial ---
# Certifique-se de que img_foto_documento e img_selfie estão definidos antes de executar
if 'img_foto_documento' in locals() and 'img_selfie' in locals() and img_foto_documento is not None and img_selfie is not None:
     face_comparison_results = perform_face_comparison(img_foto_documento, img_selfie)
     print("\n--- Resultados da Etapa de Reconhecimento Facial ---")
     print(f"Status: {'Sucesso' if face_comparison_results['success'] else 'Falha no Processo'}")
     print(f"Mensagem do Processo: {face_comparison_results['message']}")
     if face_comparison_results['success']:
        print(f"Resultado da Comparação (Match): {face_comparison_results['face_match']}")
        print(f"Distância Facial: {face_comparison_results['distance']:.4f} (Menor é mais similar. Limiar padrão ~0.6)")
     print("-------------------------------------------------")

elif 'img_foto_documento' not in locals() or img_foto_documento is None:
    print("A imagem da foto do documento não foi carregada. Execute a(s) célula(s) de upload primeiro.")
    face_comparison_results = {'success': False, 'message': 'Imagem da foto do documento não carregada.'}
elif 'img_selfie' not in locals() or img_selfie is None:
     print("A imagem da selfie não foi carregada. Execute a(s) célula(s) de upload primeiro.")
     face_comparison_results = {'success': False, 'message': 'Imagem da selfie não carregada.'}
else:
     print("Imagens necessárias para reconhecimento facial não foram carregadas. Execute a(s) célula(s) de upload primeiro.")
     face_comparison_results = {'success': False, 'message': 'Imagens essenciais não carregadas.'}

### 5. Feedback de Validação Final Combinado

Esta função recebe os resultados das etapas anteriores e fornece um veredito final e feedback detalhado.

In [ ]:
# -*- coding: utf-8 -*-

def provide_final_validation_feedback(ocr_results, face_results):
    """
    Combina os resultados do OCR/Dados e Reconhecimento Facial para dar o feedback final.

    Args:
        ocr_results (dict): Dicionário com resultados do OCR e comparação de dados.
        face_results (dict): Dicionário com resultados da comparação facial.
    """
    print("--- Resultado da Validação de Identidade ---")

    validation_status = "Inválido"
    messages = []

    # Avaliar resultados do OCR e Dados
    if not ocr_results['success']:
        messages.append(f"Falha no processamento de dados do documento: {ocr_results['message']}")
    else:
        if not ocr_results['name_match']:
            messages.append("O nome extraído do documento NÃO confere com o nome fornecido.")
        if not ocr_results['cpf_match']:
             messages.append("O CPF extraído do documento NÃO confere com o CPF fornecido.")
        if ocr_results['name_match'] and ocr_results['cpf_match']:
             messages.append("Dados de Nome e CPF do documento conferem com os dados fornecidos.")
        elif ocr_results['success'] and not (ocr_results['name_match'] or ocr_results['cpf_match']):
             messages.append("Nenhum dado chave (Nome, CPF) pôde ser confirmado a partir do documento ou não confere com os dados fornecidos.")


    # Avaliar resultados da Comparação Facial
    if not face_results['success']:
        messages.append(f"Falha no processamento facial: {face_results['message']}")
    else:
        if face_results['face_match']:
            messages.append(f"A face na selfie corresponde à face na foto do documento (Distância: {face_results['distance']:.4f}).")
        else:
            messages.append(f"A face na selfie NÃO corresponde à face na foto do documento (Distância: {face_results['distance']:.4f}).")


    # Determinar status final
    # Condição para Válido: OCR processado com sucesso + Dados de Nome E CPF conferem + Comparação facial deu match
    # Note: A lógica de validação pode ser ajustada conforme a regra de negócio.
    # Aqui, exigimos match nos dados essenciais E match facial.
    if ocr_results['success'] and ocr_results['name_match'] and ocr_results['cpf_match'] and face_results['success'] and face_results['face_match']:
         validation_status = "Válido"
         messages.append("Todas as verificações de identidade foram bem-sucedidas.")
    else:
         validation_status = "Inválido"
         messages.append("A validação de identidade falhou devido às inconsistências ou erros acima.")


    # --- Exibir o Feedback ---
    print(f"\nSTATUS FINAL: {validation_status}")
    print("\nDetalhes:")
    for msg in messages:
        print(f"- {msg}")

    print("------------------------------------------")


# --- Executar a etapa de Feedback Final ---
# Certifique-se de que ocr_data_results e face_comparison_results estão definidos
if 'ocr_data_results' in locals() and 'face_comparison_results' in locals():
     provide_final_validation_feedback(ocr_data_results, face_comparison_results)
else:
     print("Resultados das etapas anteriores (OCR e Reconhecimento Facial) não disponíveis.")
     print("Por favor, execute todas as células em ordem.")

<div style="text-align: center;"> <h3>Integração com Redes Sociais (Simulada)</h3> </div>

- **Linkagem de Contas:** Na ausência de integrações reais, simular o processo solicitando ao usuário os nomes de usuário ou perfis em redes sociais (Twitter, Facebook, Instagram). Criar células de código comentadas que demonstram como usar APIs (ex: `tweepy` para Twitter, `facebook-sdk` para Facebook) para conectar as contas.  
- **Extração de Atividades Relacionadas a e-sports:** Para cada rede social simulada, mostrar como coletar dados relevantes: por exemplo, obter os últimos *tweets* do usuário e filtrar menções a e-sports ou FURIA; listar as páginas seguidas no Facebook com temas de gaming; ou verificar hashtags usadas em posts do Instagram. Usar `requests` e `BeautifulSoup` ou clientes de API para simulação. Armazenar essas informações em DataFrames para análise.  
- **Monitoramento de Interações:** Demonstrar código que analisa curtidas, comentários ou retweets em publicações de e-sports (fingindo as permissões de API). Por exemplo, usar `tweepy` para buscar tweets que mencionem FURIA ou eventos de e-sports que o usuário retuitou ou comentou.  

<div style="text-align: center;"> <h3>Enriquecimento de Perfil com Dados Sociais e Multimídia</h3> </div>

- **Análise de Comentários:** Para integrar comentários prévios do usuário no YouTube, Reddit e Twitter, incluir blocos que consumam APIs ou dados locais de análise anterior (supondo que existam). Usar `google-api-python-client` para extrair comentários de vídeos de e-sports do YouTube, `PRAW` para posts/comentários no Reddit, e `tweepy` ou dados simulados para tweets.  
- **Processamento de Linguagem Natural:** Aplicar NLP para entender o perfil do usuário: usar bibliotecas como `transformers` ou `spaCy` para classificar sentimento, identificar tópicos ou palavras-chave frequentes nesses comentários. Por exemplo, gerar um gráfico de palavras-chave mais mencionadas em e-sports, ou uma análise de sentimento geral sobre jogos específicos.  
- **Integração de Informações:** Combinar esses insights com os interesses declarados pelo usuário. Exibir visualmente (via `matplotlib` ou `seaborn`) uma nuvem de palavras ou gráfico que mostre as categorias de e-sports mais relevantes para o perfil (baseado em interesses + análise de comentários).  
- **Perfis em Sites de e-Sports:** Permitir que o usuário insira links para seus perfis em plataformas de e-sports (como GameBattles, HLTV, Liquipedia). Usar `requests` e `BeautifulSoup` para raspar detalhes do perfil (jogos, histórico de partidas). Em seguida, aplicar um modelo de IA (ex: `transformers` BERT) para classificar se o conteúdo textual do perfil é relevante às preferências do usuário (por exemplo, buscando termos de jogos citados pelo usuário). Mostrar se há “match” entre interesses do usuário e informações do perfil scraped.  

<div style="text-align: center;"> <h3>Estruturação do Notebook</h3> </div>

- Organizar o notebook em seções claras conforme as etapas acima: **Coleta de Dados**, **Validação de Identidade**, **Integração de Redes Sociais**, **Enriquecimento com Dados Sociais**, **Conclusão**.  
- Incluir explicações breves em cada seção usando células Markdown, resumindo o objetivo daquela etapa. Combinar descrições em texto com células de código demonstrativas.  
- Sugerir bibliotecas específicas no contexto de cada etapa: por exemplo, mencionar `ipywidgets` ou `streamlit` na coleta de dados, `pytesseract`/`OpenCV` na validação de documentos, `tweepy`/`PRAW`/`BeautifulSoup` na integração social, e `transformers`/`spaCy` na análise de linguagem.

<div style="text-align: center;"> <h3>Dicas de Apresentação do Protótipo</h3> </div>

- **Formatação Atraente:** Usar cabeçalhos (`#`, `##`), listas e imagens (logotipos de e-sports, ícones de redes sociais) para tornar o notebook visualmente agradável. Células Markdown bem elaboradas ajudam na legibilidade.  
- **Interatividade:** Incluir elementos interativos (sliders, botões de upload, caixas de seleção) via `ipywidgets` para simular um fluxo real de uso. Isso torna a demonstração dinâmica mesmo no ambiente de notebook.  
- **Visualização de Dados:** Aproveitar gráficos (matplotlib, seaborn ou plotly) para mostrar perfis de interesse ou resultados das análises de comentários. Um gráfico de barras ou nuvem de palavras torna o conteúdo mais didático.  
- **Narração do Código:** Inserir comentários explicativos e outputs de exemplo que guiem o avaliador pelo processo passo a passo. Ao final, apresentar um breve resumo dos resultados obtidos para evidenciar que todos os requisitos foram atendidos.  

<div style="text-align: center;"> <h3>Conclusão</h3> </div>

Este plano garante uma implementação completa dos requisitos do desafio, integrando coleta de informações pessoais e de interesse em e-sports, validação de identidade baseada em IA, simulação de integração social e enriquecimento de perfil com dados externos. A organização em seções claras, o uso de bibliotecas especializadas (ex: **Streamlit/ipywidgets** para interfaces, **OpenCV/face_recognition** para validação, **transformers/spaCy** para IA, **pandas** para dados) e as sugestões de apresentação asseguram uma entrega alinhada e de fácil acompanhamento, mesmo no formato de notebook.